# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

SQL
``` sql
       {{ config(materialized='table') }}

   SELECT
      extract(year from order_approved_at) as order_year,
      count (order_id) as total
   FROM 
      `dbt_shop.orders` 
    WHERE
      order_approved_at IS NOT NULL
   GROUP BY 
      order_year
```
Comando terminal:</br>
`dbt run --models shop.orders_by_year`

Print </br>
![Questão 01](imagens/questao01.jpg)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

SQL
``` sql
   SELECT
       customers.customer_state as order_state,
       count(orders.order_id) as quantity_orders
   FROM
       `dbt_shop.orders` as orders
   INNER JOIN
       `dbt_shop.customers` as customers on (orders.customer_id = customers.customer_id)
   GROUP BY
       customers.customer_state
```
Comando terminal:</br>
`dbt run --models shop.total_orders_by_state`

Print </br>
![Questão 02](imagens/questao02.jpg)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

SQL
``` sql
   SELECT * FROM
       `dataset_vinicius.total_orders_by_state`
   WHERE
       quantity_orders > CAST('{{ var("number_of_orders") }}' as INT64)
```
Comando terminal:</br>
`dbt run --models shop.total_orders_by_state_variable --vars "{'number_of_orders': '5000'}"`

Print </br>
![Questão 03](imagens/questao03.jpg)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

SQL
``` sql
   SELECT 
       CASE
           WHEN customer_state LIKE 'AM' THEN 'Norte'
           WHEN customer_state LIKE 'RR' THEN 'Norte'
           WHEN customer_state LIKE 'AP' THEN 'Norte'
           WHEN customer_state LIKE 'PA' THEN 'Norte'
           WHEN customer_state LIKE 'TO' THEN 'Norte'
           WHEN customer_state LIKE 'RO' THEN 'Norte'
           WHEN customer_state LIKE 'AC' THEN 'Norte'
           WHEN customer_state LIKE 'RN' THEN 'Nordeste'
           WHEN customer_state LIKE 'CE' THEN 'Nordeste'
           WHEN customer_state LIKE 'MA' THEN 'Nordeste'
           WHEN customer_state LIKE 'PI' THEN 'Nordeste'
           WHEN customer_state LIKE 'PE' THEN 'Nordeste'
           WHEN customer_state LIKE 'PB' THEN 'Nordeste'
           WHEN customer_state LIKE 'SE' THEN 'Nordeste'
           WHEN customer_state LIKE 'AL' THEN 'Nordeste'
           WHEN customer_state LIKE 'BA' THEN 'Nordeste'
           WHEN customer_state LIKE 'SP' THEN 'Sudeste'
           WHEN customer_state LIKE 'RJ' THEN 'Sudeste'
           WHEN customer_state LIKE 'MG' THEN 'Sudeste'
           WHEN customer_state LIKE 'ES' THEN 'Sudeste'
           WHEN customer_state LIKE 'MT' THEN 'Centro Oeste'
           WHEN customer_state LIKE 'MS' THEN 'Centro Oeste'
           WHEN customer_state LIKE 'GO' THEN 'Centro Oeste'
           WHEN customer_state LIKE 'DF' THEN 'Centro Oeste'
           WHEN customer_state LIKE 'PR' THEN 'Sul'
           WHEN customer_state LIKE 'SC' THEN 'Sul'
           WHEN customer_state LIKE 'RS' THEN 'Sul'
       END AS region,
       COUNT(customer_id) AS quantity_customers
   FROM `dbt_shop.customers`
   GROUP BY region
```
Comando terminal:</br>
`dbt run --models shop.customers_by_region`

Print </br>
![Questão 04](imagens/questao04.jpg)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

SQL
``` sql
   {{ config(materialized='table') }}

   SELECT
      order_id,
      order_item_id,
      product_id,
      price + freight_value AS total_item_cost
   FROM 
      `dbt_shop.items`
```
Comando terminal:</br>
`dbt run --models shop.items_costs`

Print </br>
![Questão 05](imagens/questao05.jpg)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

SQL
``` sql
   SELECT
     orders.order_id,
     orders.order_status,
     SUM(items.price) + MAX(items.freight_value) as total_value
   FROM
     `dbt_shop.orders` as orders
     INNER JOIN `dbt_shop.items` as items ON orders.order_id = items.order_id
   WHERE
     orders.order_status like "shipped"
   GROUP BY
     orders.order_id,
     orders.order_status
```
Comando terminal:</br>
`dbt run --models shop.orders_shipped_costs`

Print </br>
![Questão 06](imagens/questao06.jpg)